In [7]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
from skimage import measure

image_folder_path = 'cell_image/im_Superficial-Intermediate/121.bmp'
svae_path = 'final_test/crop/im_Superficial-Intermediate'
save_name ='im_Superficial-Intermediate_121.bmp_crop'

#1.讀取圖片
img = cv2.imread(image_folder_path)
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

##########################################################################################
#2.分割細胞團

#rgb影像轉灰階
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#影像平滑(中值慮波)
median = cv2.medianBlur(img_gray,23)
#增強對比度(自適應直方圖均衡化)
clahe_1 = cv2.createCLAHE(clipLimit= 5.0,tileGridSize=(1,1))
cl_1 = clahe_1.apply(median)
#二質化(otsu)
ret, th_1 = cv2.threshold(cl_1, 0, 255 ,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#型態學處理(開運算)
kernel_1 = np.ones((15,15),np.uint8)
opening_1 = cv2.morphologyEx(th_1, cv2.MORPH_OPEN, kernel_1, iterations = 6) 
#使用遮罩分割細胞團
roi_1 = cv2.bitwise_and(img_rgb,img_rgb,mask = opening_1)

##########################################################################################
#3.分割細胞核

#rgb影像轉綠色通道灰階
img_g = img_rgb[:,:,1]
#影像平滑(高斯濾波)
blur_1 = cv2.GaussianBlur(img_g,(7,7),0)
#增強對比度(自適應直方圖均衡化)
clahe_2 = cv2.createCLAHE(clipLimit= 3.0,tileGridSize=(15,15))
cl_2 = clahe_2.apply(blur_1)

#百分比閾值(找出最黑的3%閥值)
def p_tile_threshold(image, pct):
    n_pixels = pct * image.shape[0] * image.shape[1]
    hist = np.histogram(image, bins=range(256))[0]
    hist = np.cumsum(hist)
    return np.argmin(np.abs(hist - n_pixels))

threshold=p_tile_threshold(cl_2,0.03)
#二值化(簡單閾值)
ret, th_2 = cv2.threshold(cl_2,threshold,255,cv2.THRESH_BINARY_INV)
#形態學處理(開運算、擴張)
kernel_3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(30,30))
kernel_4 = np.ones((5,5),np.uint8)

opening_2 = cv2.morphologyEx(th_2, cv2.MORPH_OPEN, kernel_3) 
dilation_2 = cv2.dilate(opening_2,kernel_4,iterations = 3) 
#使用遮罩分割細胞核
roi_2 = cv2.bitwise_and(img_rgb,img_rgb,mask = dilation_2)

##############################################################################################
#4.產生markers，使用標註分水嶺方法分割個別細胞

#尋找背景
sure_bg = opening_1
#尋找前景
sure_fg = dilation_2
#未知區域(用分水嶺判斷邊界)
unknown = cv2.subtract(sure_bg, sure_fg)
#類別標記
ret3, markers = cv2.connectedComponents(sure_fg)
#問為所有標記加10灰階值
markers = markers+10
#將所有未知區域的灰階值設為0
markers[unknown==255] = 0
#使用分水嶺演算法
markers = cv2.watershed(roi_1, markers)

#使用skimage.measure.regionprops找出各別細胞的屬性
props = measure.regionprops(markers)
#依據bbox裁剪細胞
for i in range(1,len(props)):
    minr, minc, maxr, maxc = props[i].bbox
    crop_img = img[minr:maxr,minc:maxc]
    cv2.imwrite(svae_path+"/"+save_name+"_"+str(i)+".jpg",crop_img)